In [1]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')

## 必要なCSVデータを読み込み

In [2]:
# 商品の売上データ
sales = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/sales_history.csv')

# 商品IDと商品カテゴリ名
category = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/category_names.csv') 

# 商品IDと商品カテゴリID
item = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/item_categories.csv')

# テストデータ
test = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/test.csv', index_col='index')

## よく使う処理を関数にまとめる

In [3]:
#  日付（yyyy-mm-dd）の「yyyy」と「mm」、「dd」に分割する関数
#  Dataframe型で返却する
def split_date(df):
    df['年'] = df['日付'].apply(lambda x: x.split('-')[0])
    df['月'] = df['日付'].apply(lambda x: x.split('-')[1])
    #df['日'] = df['日付'].apply(lambda x: x.split('-')[2])

    return df

#  売上個数を月単位で集計する関数
#  Dataframe型で返却する
def get_monthly_sales_volume(df):
    df = df.groupby(['商品ID', '店舗ID', '年', '月']).agg({'売上個数': 'sum'}).reset_index()

    return df

#  割引率を月単位で平均する関数
#  Dataframe型で返却する
def get_monthly_discount_volume(df):
    df = df.groupby(['商品ID', '店舗ID', '年', '月']).agg({'割引率': 'mean'}).reset_index()

    return df

# 商品カテゴリ名をハイフンの前後で分ける関数
# 商品カテゴリ名をsplitしたDataFrameで返却する
def split_item_category_name(df, name0='大分類', name1='小分類'):
    item_category_name_split = df.商品カテゴリ名.str.split(' - ', expand = True).rename(columns = {0 : name0, 1 : name1})
    df = pd.concat([df, item_category_name_split], axis = 1).drop('商品カテゴリ名', axis = 1)
    return df

# 訓練データとテストデータを分ける関数
# split_row = df.shape[0]
def split_dataframe(concat_df, split_row):
    train = concat_df.iloc[:split_row, :]
    test  = concat_df.iloc[split_row:, :]

    return train,test

# 年月の型を変換する関数
# Dataframe型で返却する
def to_int_year_month(df):
    df['年'] = df['年'].astype('int')
    df['月'] = df['月'].astype('int')
    return df

In [4]:
# ターゲット170商品にしぼる関数
# Dataframeで返却する＆ファイル保存も行う
def output_target_item(df, test,filename='target_item_170'):
    item_id_list = test['商品ID'].unique()
    df = df[df['商品ID'].isin(item_id_list)]

    # 無いデータの補完
    #すべての組み合わせ（全商品×全店舗×全ての年×全ての月）を検討するために全種類を変数に取り出す
    item_id_list  = df["商品ID"].unique()
    store_id_list = df["店舗ID"].unique()

    year_list = [2018, 2019]
    month_list = [1,2,3,4,5,6,7,8,9,10,11,12] # 1~12月

    zero_table = pd.DataFrame()
    #全商品（170）×全店舗（18）×全ての年（2）×全ての月（12）についてfor文を回す
    for item_id in item_id_list:
        for store_id in store_id_list:
            for year in year_list:
                for month in month_list:
                    if year == 2019 and month == 11:
                        break
                    cat = math.floor(item_id/10000)
                    line = pd.DataFrame({"商品ID" : item_id, "店舗ID" : store_id, "年" : year, "月" : month, "売上個数" : 0, "商品カテゴリID" : cat}, index = [1])
                    zero_table = pd.concat([zero_table,line],axis = 0)

    df = pd.concat([df,zero_table], axis = 0)
    df = df.drop_duplicates(subset = ["商品ID", "店舗ID", "年", "月", "商品カテゴリID"], keep = "first")

    #残った追加予定の売上個数0のデータは下に固まっているためsort_valuesを使って整列させる
    df = df.sort_values(["商品ID", "店舗ID", "年", "月"])
    df = df.reset_index()

    # ファイルを保存する
    df.to_csv(filename + '.csv', index = False)    

    print('{}を保存しました。'.format(filename))
    df = df.drop('index', axis=1)

    return df

## 前処理
訓練データとテストデータをDataFrameを整備する

### 売上データを月単位で集計

In [5]:
# 売上データの売上個数を月単位で集計する
split_date_sales_df = split_date(sales)
monthly_sales_volume_df = get_monthly_sales_volume(split_date_sales_df)
monthly_sales_volume_df.head(2)

,商品ID,店舗ID,年,月,売上個数
0,1000001,0,2018,01,6.0
1,1000001,0,2018,02,3.0


In [6]:
# 商品カテゴリIDをDataFrameの列に追加する
monthly_sales_df = pd.merge(monthly_sales_volume_df, item, on = '商品ID')
monthly_sales_df.head()

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID
0,1000001,0,2018,01,6.0,100
1,1000001,0,2018,02,3.0,100
2,1000001,0,2018,03,1.0,100
3,1000001,0,2018,04,2.0,100
4,1000001,0,2018,06,4.0,100


### 商品IDと店舗IDごとの割引を出したい！

- 定価を出す
- 日単位で割引率を計算する。
- 月単位で割引率を集計する。

In [7]:
# 商品IDごとの定価は最大値とする。
# sales :売上データ（CSVから読み込んだもの）
# item_max_price : 定価を含んだDataFrame
item_max_price = sales.groupby(['商品ID']).agg({'商品価格' : 'max'}).rename(columns={'商品価格' : '定価'})
item_max_price = pd.merge(sales, item_max_price, on = '商品ID')

item_max_price.tail(40)

,日付,店舗ID,商品ID,商品価格,売上個数,年,月,定価
1119530,2019-10-31,14,1002619,680,1.0,2019,10,680
1119531,2019-10-31,0,1002619,680,2.0,2019,10,680
1119532,2019-10-31,13,1002619,680,1.0,2019,10,680
1119533,2019-10-29,6,1201332,510,1.0,2019,10,510
1119534,2019-10-29,7,1201332,510,1.0,2019,10,510
1119535,2019-10-30,11,1201332,510,1.0,2019,10,510
1119536,2019-10-30,6,1201332,510,1.0,2019,10,510
1119537,2019-10-31,8,1201332,510,1.0,2019,10,510
1119538,2019-10-31,13,1201332,510,1.0,2019,10,510
1119539,2019-10-29,0,2100175,1090,1.0,2019,10,1090


In [8]:
# 商品価格/定価で割引率を計算する。
# 下2桁のみ表示する
item_max_price['割引率'] = round((item_max_price['商品価格'] / item_max_price['定価']), 2)
item_max_price.head(10)

,日付,店舗ID,商品ID,商品価格,売上個数,年,月,定価,割引率
0,2018-01-01,9,1000001,420,1.0,2018,01,420,1.00
1,2018-01-01,6,1000001,420,1.0,2018,01,420,1.00
2,2018-01-01,10,1000001,420,1.0,2018,01,420,1.00
3,2018-01-02,10,1000001,420,2.0,2018,01,420,1.00
4,2018-01-02,9,1000001,420,1.0,2018,01,420,1.00
5,2018-01-02,15,1000001,420,1.0,2018,01,420,1.00
6,2018-01-02,6,1000001,420,1.0,2018,01,420,1.00
7,2018-01-02,16,1000001,420,1.0,2018,01,420,1.00
8,2018-01-02,2,1000001,420,1.0,2018,01,420,1.00
9,2018-01-02,5,1000001,340,1.0,2018,01,420,0.81


In [9]:
# 割引率を月単位で平均する
monthly_discount_volume = get_monthly_discount_volume(item_max_price)
monthly_discount_volume.head(2)

,商品ID,店舗ID,年,月,割引率
0,1000001,0,2018,01,1.00
1,1000001,0,2018,02,0.85


In [10]:
# 集計した月単位のDataFrameに月単位の割引率を追加する
# 月単位の売上データ：monthly_sales_df
# 月単位の割引率　　：monthly_discount_volume['割引率']
monthly_df = pd.concat([monthly_sales_df, monthly_discount_volume['割引率']], axis = 1)

# DataFrameの年と月をint型に変換する
monthly_df = to_int_year_month(monthly_df)
monthly_df.head(2)

# 確認用のCSVを吐き出すコード
# monthly_df.to_csv(BASE_INPUT_PATH + 'discount' + '.csv', index = False)

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID,割引率
0,1000001,0,2018,1,6.0,100,1.00
1,1000001,0,2018,2,3.0,100,0.85


In [11]:
# 商品ID・店舗IDごとの割引率で表示したい
item_price_discount = item_max_price.drop(['日付','商品価格', '年', '月'], axis=1)

item_price = item_price_discount.drop_duplicates()
item_price.head()

,店舗ID,商品ID,売上個数,定価,割引率
0,9,1000001,1.0,420,1.0
1,6,1000001,1.0,420,1.0
2,10,1000001,1.0,420,1.0
3,10,1000001,2.0,420,1.0
5,15,1000001,1.0,420,1.0


In [12]:
# 割引率の月平均を抽出
# 商品ID/店舗ID/定価をキーに割引率を（平均で）計算する
store_item_discount_df = item_price.groupby(['商品ID', '店舗ID', '定価']).agg({'割引率' : 'mean'}).rename(columns={'割引率' : '商品店舗ごとの割引率'})
store_item_discount_df

商品店舗ごとの割引率
商品ID    店舗ID 定価             
1000001 0    420      0.7875
        1    420      0.7750
        2    420      0.8000
        3    420      0.8000
        4    420      0.8000
...                      ...
3500070 16   420      1.0000
3500071 13   590      1.0000
3500072 2    420      1.0000
        16   420      1.0000
3500073 17   420      1.0000

[107115 rows x 1 columns]

## ターゲットを１７０種類に絞る
- 商品IDから商品170種類を抽出し、リスト化する
- リスト化したターゲットの商品をDataFrameに格納する

In [13]:
# ターゲット170種類に絞ったときのCSVファイル名
filename = 'target_item_170'

# 商品170種類を抽出
item_id_list = test['商品ID'].unique()

# 対象の170商品のみDataFrameに格納
monthly_sales_df = monthly_df[monthly_df['商品ID'].isin(item_id_list)]

In [14]:
monthly_sales_df

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID,割引率
0,1000001,0,2018,1,6.0,100,1.00
1,1000001,0,2018,2,3.0,100,0.85
2,1000001,0,2018,3,1.0,100,1.00
3,1000001,0,2018,4,2.0,100,1.00
4,1000001,0,2018,6,4.0,100,1.00
...,...,...,...,...,...,...,...
492928,3500001,14,2018,6,1.0,350,1.00
492929,3500001,15,2019,5,1.0,350,0.48
492930,3500001,16,2018,9,1.0,350,1.00
492931,3500001,16,2019,3,1.0,350,1.00


In [1]:
# fileを作成& Dataframe取得
target_item_170 = output_target_item(monthly_sales_df, test, filename)

NameError: name 'output_target_item' is not defined

## 訓練データとテストデータを作成する

In [17]:
# 訓練データの作成
# target_item_170 : 170種類に絞ったDataFrame
# category : category_names.csvのデータ
target_item = pd.merge(target_item_170, category, on = '商品カテゴリID')
df = split_item_category_name(target_item)

df.head()

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID,割引率,大分類,小分類
0,1000001,0,2018,1,6.0,100,1.00,映画,DVD
1,1000001,0,2018,2,3.0,100,0.85,映画,DVD
2,1000001,0,2018,3,1.0,100,1.00,映画,DVD
3,1000001,0,2018,4,2.0,100,1.00,映画,DVD
4,1000001,0,2018,5,0.0,100,NaN,映画,DVD


In [18]:
# 訓練用のDataFrameをMergeする
# df : 訓練データのもとになるDataFrame
# store_item_discount_df : 商品IDと店舗IDごとの割引率のDataFrame
df_add_discount_tmp = pd.merge(df, store_item_discount_df, on = ['商品ID', '店舗ID'], how = 'left')

# 売上個数が最大の行を抽出＆削除する
target = df_add_discount_tmp[df_add_discount_tmp['売上個数'] == df_add_discount_tmp['売上個数'].max()].index[0]

df = df_add_discount_tmp.drop(target)
df.head()

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID,割引率,大分類,小分類,商品店舗ごとの割引率
0,1000001,0,2018,1,6.0,100,1.00,映画,DVD,0.7875
1,1000001,0,2018,2,3.0,100,0.85,映画,DVD,0.7875
2,1000001,0,2018,3,1.0,100,1.00,映画,DVD,0.7875
3,1000001,0,2018,4,2.0,100,1.00,映画,DVD,0.7875
4,1000001,0,2018,5,0.0,100,NaN,映画,DVD,0.7875


In [19]:
# testデータの作成
# 訓練データと同じカラムの形式にしていく

# 「年」カラムの追加
test['年'] = 2019
# 「月」カラムの追加
test['月'] = 12

test_add_item = pd.merge(test, item, on = '商品ID', how = 'inner')
test_add_item_category_id = pd.merge(test_add_item, category, on = '商品カテゴリID')

test_df = split_item_category_name(test_add_item_category_id)
test_df.head()

,商品ID,店舗ID,年,月,商品カテゴリID,大分類,小分類
0,1000001,0,2019,12,100,映画,DVD
1,1000001,1,2019,12,100,映画,DVD
2,1000001,2,2019,12,100,映画,DVD
3,1000001,3,2019,12,100,映画,DVD
4,1000001,4,2019,12,100,映画,DVD


In [20]:
# テストデータに商品店舗ごとの割引率を追加する
test_col = ['商品店舗ごとの割引率']
test = pd.merge(test_df, store_item_discount_df[test_col], on = ['商品ID', '店舗ID'], how = 'left')
test.head()

,商品ID,店舗ID,年,月,商品カテゴリID,大分類,小分類,商品店舗ごとの割引率
0,1000001,0,2019,12,100,映画,DVD,0.7875
1,1000001,1,2019,12,100,映画,DVD,0.7750
2,1000001,2,2019,12,100,映画,DVD,0.8000
3,1000001,3,2019,12,100,映画,DVD,0.8000
4,1000001,4,2019,12,100,映画,DVD,0.8000


## データの確認
機械学習モデルに学習させる前にデータを確認する

In [21]:
# データの個数を確認
print('訓練データ:{}行、{}列'.format(df.shape[0], df.shape[1]))
print('テストデータ:{}行、{}列'.format(test.shape[0], test.shape[1]))

訓練データ:67319行、10列
テストデータ:3060行、8列


In [22]:
df.head(5)

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID,割引率,大分類,小分類,商品店舗ごとの割引率
0,1000001,0,2018,1,6.0,100,1.00,映画,DVD,0.7875
1,1000001,0,2018,2,3.0,100,0.85,映画,DVD,0.7875
2,1000001,0,2018,3,1.0,100,1.00,映画,DVD,0.7875
3,1000001,0,2018,4,2.0,100,1.00,映画,DVD,0.7875
4,1000001,0,2018,5,0.0,100,NaN,映画,DVD,0.7875


In [23]:
test.head()

,商品ID,店舗ID,年,月,商品カテゴリID,大分類,小分類,商品店舗ごとの割引率
0,1000001,0,2019,12,100,映画,DVD,0.7875
1,1000001,1,2019,12,100,映画,DVD,0.7750
2,1000001,2,2019,12,100,映画,DVD,0.8000
3,1000001,3,2019,12,100,映画,DVD,0.8000
4,1000001,4,2019,12,100,映画,DVD,0.8000


## モデルに学習させる

In [24]:
# 説明変数に該当するカラムのリスト
feature_columns = ['商品ID', '店舗ID', '商品カテゴリID', '年', '月', '大分類', '小分類']

In [25]:
# 学習用データの整理
X_train = df[feature_columns] # 学習用データの説明変数
y_train = df['売上個数'] # 学習用データの目的変数

# テスト用データの整理
X_test = test[feature_columns] # テスト用データの説明変数

In [26]:
# ont-hot encordingの対象
cat_features = ['大分類', '小分類']

# one-hot encording
train_df = pd.get_dummies(X_train[feature_columns], drop_first = True)
test_df = pd.get_dummies(X_test[feature_columns], drop_first = True)

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

# 交差検証用
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

In [28]:
# 評価関数RMSE
def rmse(y_true, y_pred):
    #RMSEを算出
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print('rmse', rmse)
    
    return rmse

## モデルの生成

In [29]:
# モデルの型の生成
from xgboost import XGBRegressor

# モデルのインスタンスを作成する
xgb = XGBRegressor(random_state = 7)

# パラメータ
xgb_params = {
    'max_depth': [i for i in range(3,6,1)],
    'learning_rate' : [0.05, 0.1],
    'reg_lambda' : [0.001, 0.05],
    'booster' : ['dart', 'gbtree', 'gbliner'],
    'eval_metric' : ['rmse'],
    'gamma' : [0.001, 0.01],
}

# パラメータチューニング
# estimator : モデルのインスタンス
# param_grid : パラメータの探索範囲
# scoring : スコアを出す評価関数
# cv : 交差検証の回数
xgb_cv = GridSearchCV(estimator = xgb, param_grid = xgb_params, scoring = make_scorer(rmse, greater_is_better = False), cv = kf)

# 評価
xgb_cv.fit(train_df, y_train)

# スコアが最も良かったときのパラメータを表示
print('ハイパーパラメータは、{}です。'.format(xgb_cv.best_params_))
print('ベストスコアは、{}です。'.format(-xgb_cv.best_score_))

rmse 1.4735852471206345
rmse 1.5596533057917876
rmse 1.4456173665984027
rmse 1.4912402232459914
rmse 1.7703695788649485
rmse 1.4736218118422895
rmse 1.5607281661933756
rmse 1.4453472107860874
rmse 1.489279840324332
rmse 1.7703954292647734
rmse 1.4193140691037545
rmse 1.5032345623226124
rmse 1.3813565410502533
rmse 1.4347231592592715
rmse 1.7440979242760355
rmse 1.4211796029590213
rmse 1.5073980119149628
rmse 1.3831959608960611
rmse 1.4328588697104834
rmse 1.7394355478200993
rmse 1.3417275095202421
rmse 1.4557795857643065
rmse 1.3129604905543895
rmse 1.3673370659000406
rmse 1.6829093168411369
rmse 1.3456713171524624
rmse 1.4682864352196614
rmse 1.3190830651961283
rmse 1.358504628357124
rmse 1.6738735974125203
rmse 1.422633471753186
rmse 1.518492957793491
rmse 1.4031357531393225
rmse 1.443458599310914
rmse 1.7379936732090713
rmse 1.4226755841378618
rmse 1.5181971831943817
rmse 1.4031562540499114
rmse 1.4434909475946331
rmse 1.7380237541124555
rmse 1.3567318575094383
rmse 1.44476699783103

In [30]:
# best_params_で表示されたパラメータを代入
xgb_best = XGBRegressor(
    max_depth = xgb_cv.best_params_['max_depth'],
    learning_rate = xgb_cv.best_params_['learning_rate'],
    reg_lambda = xgb_cv.best_params_['reg_lambda'],
    booster = xgb_cv.best_params_['booster'],
    eval_metric = xgb_cv.best_params_['eval_metric'],
    gamma = xgb_cv.best_params_['gamma'],
    random_state = 0
)

# 評価
xgb_best.fit(train_df, y_train)

# テストデータの売上個数を予測する
y_pred = xgb_best.predict(test_df)
y_pred = y_pred.round()

In [51]:
y_pred.mean()

1.677124183006536

### 提出用CSVの準備

In [32]:
# CSV読み込み
submission = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/sample_submission.csv', header = None)

# sample_submissionの右端のカラムに予測値を代入する。
submission.iloc[:, -1] = y_pred

# 予測した売上個数がマイナスの場合、0個として扱う
submission.loc[submission[1] < 0, 1] = 0

# データの確認
submission.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
1,1.0,0.0,1.0,6.0,6.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0


In [33]:
# ファイルの名前
FILENAME = 'submit_20221006'

# フォルダのパス＋ファイルの名前
FILEPATH_en1 = FILENAME + '.csv'
submission.to_csv(FILEPATH_en1, index = False, header = False)

## lightGBMを組んでみる

In [34]:
from lightgbm import LGBMRegressor 

# モデルのインスタンスを作成する
lgb = LGBMRegressor(random_state = 0)

# パラメータ
lgb_params = {
    'boosting_type' : ['gbdt', 'dart', 'goss', 'rf'],
    'objective': ['regression'],
    'metric' : ['rmse'],
    'max_depth' : [i for i in range(4,7, 1)],
    'num_leaves' : [i for i in range(15,25,5)],
    'reg_lambda' : np.logspace(-3,-1, num=2)
}

# パラメータチューニング
# estimator : モデルのインスタンス
# param_grid : パラメータの探索範囲
# scoring : スコアを出す評価関数
# cv : 交差検証の回数
lgb_cv = GridSearchCV(estimator = lgb, param_grid = lgb_params,scoring = make_scorer(rmse, greater_is_better = False), cv = kf)

# 評価
lgb_cv.fit(train_df, y_train)

lgb_cv.best_params_


rmse 1.3646468893321446
rmse 1.4516025067525917
rmse 1.3364132926776726
rmse 1.3811413800589518
rmse 1.6742174203211089
rmse 1.367894971491074
rmse 1.4689793550221948
rmse 1.3312697423588389
rmse 1.3730852376157188
rmse 1.6803666739088525
rmse 1.370318683634343
rmse 1.46734230452849
rmse 1.3306259855563358
rmse 1.3742721316024644
rmse 1.689978454682356
rmse 1.3749026862387534
rmse 1.4529449183136016
rmse 1.3408258112902522
rmse 1.375652595380603
rmse 1.67929309046792
rmse 1.3209053849267876
rmse 1.4108804081070256
rmse 1.2981798470108818
rmse 1.3419984648390852
rmse 1.6188438256351256
rmse 1.3046982318707567
rmse 1.4329502419872
rmse 1.2856213248584736
rmse 1.3399253215984392
rmse 1.6270258600286294
rmse 1.3032178883809877
rmse 1.4016060139157425
rmse 1.2742124705760363
rmse 1.3373632356936815
rmse 1.6397612730291518
rmse 1.3204100815034299
rmse 1.4036129534920236
rmse 1.2762526287949902
rmse 1.333460993963814
rmse 1.6256740882416187
rmse 1.3039136732524272
rmse 1.3922154616678628
rmse

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && con

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /Users/runner/miniforge3/conda-bld/lightgbm_1641600054035/work/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && con

{'boosting_type': 'gbdt',
 'max_depth': 6,
 'metric': 'rmse',
 'num_leaves': 20,
 'objective': 'regression',
 'reg_lambda': 0.001}

In [36]:
# best_params_で表示されたパラメータを代入
lgb_best = LGBMRegressor(
    boosting_type = lgb_cv.best_params_['boosting_type'],
    max_depth = lgb_cv.best_params_['max_depth'],
    metric = lgb_cv.best_params_['metric'],
    num_leaves = lgb_cv.best_params_['num_leaves'],
    objective = lgb_cv.best_params_['objective'],
#    reg_lambda = lgb_cv.best_params_['reg_lambad'],
    random_state = 0
)


# 評価
lgb_best.fit(train_df, y_train)

# テストデータの売上個数を予測する
y_pred = lgb_best.predict(test_df)
y_pred = y_pred.round()

In [52]:
y_pred.mean()

1.677124183006536

### 提出用CSVの準備

In [37]:
# CSV読み込み
submission = pd.read_csv('/Users/Desktop/PBL01_1/課題3/data/sample_submission.csv', header=None)

# sample_submissionの右端のカラムに予測値を代入する。
submission.iloc[:, -1] = y_pred

# 予測した売上個数がマイナスの場合、0個として扱う
submission.loc[submission[1] < 0, 1] = 0

submission.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
1,2.0,1.0,2.0,5.0,4.0,2.0,2.0,3.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0


#### 提出ファイルの生成

In [38]:
# ファイルの名前
FILENAME = 'submit_20221006_2'

# フォルダのパス＋ファイルの名前
FILEPATH_en2 = FILENAME + '.csv'
submission.to_csv(FILEPATH_en2, index = False, header = False)

## シンプルなアンサンブルをやってみる
1. 上記で学習させたモデル2つをアンサンブルする
2. 既に出力したCSVでアンサンブルする

In [39]:
# アンサンブル用に学習されたDataFrameを平均する関数
def get_concat_df_mean(df):
    mean_df = pd.DataFrame(np.mean(df, axis = 1), columns = ['mean_col'])
 
    return mean_df

### 上記で学習させたモデル２つをアンサンブルする

In [40]:
en1 = pd.read_csv(FILEPATH_en1, header = None, index_col = 0).rename(columns = {1 : 'en1'})
en2 = pd.read_csv(FILEPATH_en2, header = None, index_col = 0).rename(columns = {1 : 'en2'})

ensemble_concat = pd.concat([en1, en2], axis = 1).astype(np.float64)

In [41]:
ensemble_concat

,en1,en2
0,,
0,1.0,2.0
1,0.0,1.0
2,1.0,2.0
3,6.0,5.0
4,6.0,4.0
...,...,...
3055,2.0,2.0
3056,0.0,1.0
3057,0.0,1.0


In [53]:
#submission.iloc[:, -1] = get_concat_df_mean(ensemble_concat)

submission.iloc[:, -1]  = (ensemble_concat["en1"] + ensemble_concat["en2"]*2)/3

submission.to_csv('/Users/Desktop/PBL01_1/課題3/data/sample_ensemble.csv', index = False, header = False)
submission.describe().T

,count,mean,std,min,25%,50%,75%,max
0,3060.0,1529.500000,883.490238,0.0,764.75,1529.5,2294.25,3059.000000
1,3060.0,1.691285,1.546130,0.0,1.00,1.0,2.00,19.333333
